In [20]:
from dotenv import load_dotenv
import github as gh
import bs4 as bs
import requests
from lxml import etree

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

In [2]:
load_dotenv()
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)

In [4]:
print(g.get_user())

AuthenticatedUser(login=None)


In [3]:
query='tensorflow language:python created:2021-04-01..2021-04-02'
result=g.search_repositories(query)

In [12]:
print(result.totalCount)

29


In [7]:
print(dir(result))

['_PaginatedListBase__elements', '_PaginatedListBase__fetchToIndex', '_PaginatedList__contentClass', '_PaginatedList__firstParams', '_PaginatedList__firstUrl', '_PaginatedList__headers', '_PaginatedList__list_item', '_PaginatedList__nextParams', '_PaginatedList__nextUrl', '_PaginatedList__parseLinkHeader', '_PaginatedList__requester', '_PaginatedList__reverse', '_PaginatedList__totalCount', '_Slice', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_couldGrow', '_fetchNextPage', '_getLastPageUrl', '_grow', '_isBiggerThan', '_reversed', 'get_page', 'reversed', 'totalCount']


In [14]:
for repository in result:
    # print(dir(repository))
    print(repository.html_url)


https://github.com/Z-yq/TensorflowTTS
https://github.com/Kal213/StarGAN-Tutorial-Tensorflow-2.3
https://github.com/lshug/first-order-model-tf
https://github.com/veeenie/Tensorflow2
https://github.com/hubertkarbowy/awd_lstm_tensorflow2
https://github.com/boa50/tensorflow-styletransfer
https://github.com/sk981102/tensorflow_certificate
https://github.com/fadhil-code/TensorFlowProject3
https://github.com/danilyef/ResNet-Tensorflow
https://github.com/Developik/tensorflow_image_model
https://github.com/Zwyywz/TensorFlowLearningNotes
https://github.com/boa50/tensorflow-pix2pix
https://github.com/Astro-Astre/tensorflow_merger_classifier
https://github.com/aysedemirel/TensorFlow-Pokemon-Course
https://github.com/huww98/torch-tensorflow-wrapper
https://github.com/junyuchen245/Semi-supervised_FCM_Loss_for_Segmentation
https://github.com/artset/image-to-illustration
https://github.com/Vezqi/tf-emotes
https://github.com/wchen777/ASL-recog
https://github.com/The-bot-makers/Neural-Networks
https://g

In [4]:
url='https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras'
url2='https://github.com/bamblebam/image-classification-rps'

In [5]:
options=Options()
options.headless=True
driver=webdriver.Chrome(ChromeDriverManager(path='./').install(),options=options)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [./\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [6]:
driver.get(url2)

In [7]:
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")

In [9]:
links=table.find_elements_by_tag_name('a')
print(len(links))

8


In [12]:
stack=list()
full_list=list()

In [13]:
for link in links:
    href=link.get_attribute('href')
    if 'tree' in href or '.py' in href:
        stack.append(href)

In [14]:
print(stack)

['https://github.com/bamblebam/image-classification-rps/tree/master/models', 'https://github.com/bamblebam/image-classification-rps/blob/master/image_classification.py']


In [ ]:
while stack:
    href=stack.pop(0)
    if 'tree' in href:
        

In [ ]:
stack=list()
full_list=list()

In [ ]:
driver.get(url2)
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
links=table.find_elements_by_tag_name('a')
for link in links:
    href=link.get_attribute('href')
    if '/tree/' in href or '.py' in href:
        stack.append(href)

In [53]:
def push_to_stack(links):
    for link in links:
        href=link.get_attribute('href')
        if '/tree/' in href or '.py' in href:
            stack.append(href)

In [36]:
def search_through_files(link):
    if '/tree/' in link:
        driver.get(link)
        time.sleep(2.5)
        table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-block']")
        links=table.find_elements_by_tag_name('a')
        push_to_stack(links)
    elif '.py' in link:
        driver.get(link)
        time.sleep(2.5)
        full_list.append(link)
        #preprocessing 

In [23]:
def dfs():
    while stack:
        link=stack.pop(0)
        search_through_files(link)

In [54]:
stack=list()
full_list=list()

In [45]:
def get_all_relevant_links(url):
    driver.get(url)
    table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
    links=table.find_elements_by_tag_name('a')
    for link in links:
        href=link.get_attribute('href')
        if '/tree/' in href or '.py' in href:
            stack.append(href)
    dfs()

In [55]:
res=get_all_relevant_links('https://github.com/Z-yq/TensorflowTTS')

KeyboardInterrupt: 

In [56]:
print(full_list)

['https://github.com/Z-yq/TensorflowTTS/blob/main/fastspeech_extract_features.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/run-test.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/tacotron_extract_features.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/train_acoustic.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/train_vocoder.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/fastspeech_dataloader.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/tacotron_dataloader.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/vocoder_dataloader.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/models/__init__.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/models/conformer.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/models/fastspeech.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/models/model.py', 'https://github.com/Z-yq/TensorflowTTS/blob/main/models/positional_encoding.py', 'https://github.c

In [57]:
list_set=set(full_list)
print(len(full_list),len(list_set))

125 51
